In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score, mean_absolute_percentage_error, explained_variance_score, max_error
pd.set_option('display.max_columns', None)
from IPython.display import display
import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_pinball_loss


data = pd.read_csv("preprocessed_data.csv")
data.head()

,Unnamed: 0,title,link,description,garage,price,home_type,city,home_size,home_area,floor,elevator,price_per_sqr_meter,date,neighborhood,price_per_sqr_meter_bin,price_bin,home_area_bin,street_names
0,0,apartamento t0 em sao joao da caparica costa d...,https://www.idealista.pt/imovel/33182313/,"Apartamento T0, com vista de Mar.\nNão perca a...",False,1250,apartamento,almada,t0,60,10,True,20.833333,13-05-2024,costa de caparica,"(20.64, 24.61]","(355.38, 1364.8]","(42.28, 70.56]",sao joao da caparica costa de caparica
1,1,apartamento t0 em sao joao da caparica costa d...,https://www.idealista.pt/imovel/33225472/,Não perca a oportunidade de viver num apartame...,False,1200,apartamento,almada,t0,57,6,True,21.052632,13-05-2024,costa de caparica,"(20.64, 24.61]","(355.38, 1364.8]","(42.28, 70.56]",sao joao da caparica costa de caparica
2,2,apartamento t0 na avenida da liberdade 5 capar...,https://www.idealista.pt/imovel/33132721/,"Excelente apartamento t1 na Trafaria, a 2 minu...",False,1200,apartamento,almada,t0,50,1,False,24.000000,13-05-2024,almada,"(20.64, 24.61]","(355.38, 1364.8]","(42.28, 70.56]",avenida da liberdade 5 caparica
3,3,apartamento t0 na rua eleuterio teixeira 8 cap...,https://www.idealista.pt/imovel/33394837/,"Estúdio com muita luz, casa de banho com duche...",False,1000,apartamento,almada,t0,50,0,False,20.000000,13-05-2024,almada,"(16.67, 20.64]","(355.38, 1364.8]","(42.28, 70.56]",almada
4,4,apartamento t1 em almada,https://www.idealista.pt/imovel/30954625/,Disponível a partir de 01 de Junho de 2024.\nE...,False,925,apartamento,almada,t1,65,8,True,14.230769,13-05-2024,em almada,"(12.701, 16.67]","(355.38, 1364.8]","(42.28, 70.56]",almada


In [2]:
selected_features = ["home_type", "garage", "home_size", "floor", "elevator", "city", "home_area"]
target = ["price"]

data = data[selected_features + target]

In [3]:
for c in data.columns:
    col_type = data[c].dtype
    if col_type == 'object' or col_type == 'bool':
        data[c] = data[c].astype('category')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2454 entries, 0 to 2453
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   home_type  2454 non-null   category
 1   garage     2454 non-null   category
 2   home_size  2454 non-null   category
 3   floor      2454 non-null   int64   
 4   elevator   2454 non-null   category
 5   city       2454 non-null   category
 6   home_area  2454 non-null   int64   
 7   price      2454 non-null   int64   
dtypes: category(5), int64(3)
memory usage: 71.1 KB


In [4]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=target), 
                                                    data[target], 
                                                    test_size=0.25, 
                                                    random_state=42)

In [5]:
lower = lgb.LGBMRegressor(objective = 'quantile', alpha = 1 - 0.95)
lower.fit(X_train, y_train)
lower_pred = lower.predict(X_test)

upper = lgb.LGBMRegressor(objective = 'quantile', alpha = 0.95)
upper.fit(X_train, y_train)
upper_pred = upper.predict(X_test)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000375 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 298
[LightGBM] [Info] Number of data points in the train set: 1840, number of used features: 7
[LightGBM] [Info] Start training from score 789.500000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000117 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 298
[LightGBM] [Info] Number of data points in the train set: 1840, number of used features: 7
[LightGBM] [Info] St

In [6]:
print(mean_pinball_loss(y_test, lower_pred, alpha=0.05))
print(mean_pinball_loss(y_test, upper_pred, alpha=0.95))

65.57052554087649
132.51365586570225
